In [1]:
from cloudsafe import *
from utils.folium_utils import get_geojson_grid
database = 'here-api/main.db'
conn = create_connection(database)
stations = select_stations_join_departures(conn)

In [2]:
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)
stations[['latitude', 'longitude']] = stations.apply(get_coords, axis=1, result_type="expand")

In [3]:
frequencies = stations.name_overpass.value_counts()

In [4]:
stations

,id,id_overpass,name_overpass,coordinates_overpass,id_here,name_here,coordinates_here,no_data,duplicate,id,station_id,bus,headsign,day,time,latitude,longitude
0,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0,0,1,1,12,Islev Viemosevej,saturday,00:53,55.699627,12.469424
1,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0,0,2,1,12,Islev Viemosevej,saturday,06:53,55.699627,12.469424
2,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0,0,3,1,12,Islev Viemosevej,saturday,07:53,55.699627,12.469424
3,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0,0,4,1,12,Islev Viemosevej,saturday,08:23,55.699627,12.469424
4,1,node/7732035,None,"55.699612,12.469472",205079581,Islev St. (Slotsherrensvej),"55.699627,12.469424",0,0,5,1,12,Islev Viemosevej,saturday,08:53,55.699627,12.469424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284966,1047,node/7178922089,Korsager Allé,"55.708405,12.477656",205079022,Korsager Allé (Husumvej),"55.708367,12.477652",0,0,284967,1047,22,Glostrup St. Glosemosevej,weekday,21:33,55.708367,12.477652
284967,1047,node/7178922089,Korsager Allé,"55.708405,12.477656",205079022,Korsager Allé (Husumvej),"55.708367,12.477652",0,0,284968,1047,22,Glostrup St. Glosemosevej,weekday,22:05,55.708367,12.477652
284968,1047,node/7178922089,Korsager Allé,"55.708405,12.477656",205079022,Korsager Allé (Husumvej),"55.708367,12.477652",0,0,284969,1047,22,Glostrup St. Glosemosevej,weekday,22:26,55.708367,12.477652
284969,1047,node/7178922089,Korsager Allé,"55.708405,12.477656",205079022,Korsager Allé (Husumvej),"55.708367,12.477652",0,0,284970,1047,22,Glostrup St. Glosemosevej,weekday,23:05,55.708367,12.477652


In [5]:
frequencies

Peter Bangs Vej                   3051
Polititorvet                      2716
Anker Jørgensens Plads            2588
Mozarts Plads                     2580
Elmegade                          2534
                                  ... 
Tietgensgade (Bernstorffsgade)      12
Vasbygade (Sydhavns Plads)          12
Sightseeingbus                      11
Prags Boulevard                      9
Kløvermarksvej                       8
Name: name_overpass, Length: 414, dtype: int64

In [ ]:
rows = []
for station in frequencies.keys():
    first = stations.loc[stations['name_overpass'] == station].iloc[0]
    rows.append({'station_name': station,
                 'longitude': first.longitude,
                 'latitude': first.latitude,
                 'bus_count': frequencies.get(station)})

final = pd.DataFrame(rows)
final['longitude'] = final['longitude'].astype(float)
final['latitude'] = final['latitude'].astype(float)
top_right = [final['latitude'].max(), final['longitude'].max()]
bottom_left = [final['latitude'].min(), final['longitude'].min()]

In [ ]:
# Generating a grid heatmap (utils/folium_utils.py)
grid = get_geojson_grid(top_right, bottom_left, n=10)

In [ ]:
grid

In [ ]:
# Frequencies by latitude and longitude
counts = []
for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]
    items = final.loc[(final['latitude'] <= upper_right[1]) &
                      (final['longitude'] <= upper_right[0]) &
                      (final['longitude'] >= lower_left[0]) &
                      (final['latitude'] >= lower_left[1])]
    counts.append(len(items))
max_count = max(counts)

In [ ]:
# Add GeoJson to map
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(counts[i] / max_count)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color': "black",
                            'weight': 2,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.55,
                        })

    m.add_child(gj)

In [ ]:
m